In [ ]:
import os

import pandas as pd
from elasticsearch import Elasticsearch
from tqdm.notebook import tqdm
import json

In [ ]:
df = pd.read_json("../../data/stories.json")

In [ ]:
es = Elasticsearch(
    os.environ["ELASTIC_HOST"],
    http_auth=(os.environ["ELASTIC_USERNAME"], os.environ["ELASTIC_PASSWORD"]),
)

In [ ]:
INDEX_NAME = "stories"

In [ ]:
with open('../../data/stories/mapping.json', 'r') as f:
    mappings = json.load(f)

with open('../../data/stories/settings.json', 'r') as f:
    settings = json.load(f)

In [ ]:
es.indices.delete(index=INDEX_NAME, ignore=404)

In [ ]:
es.indices.create(index=INDEX_NAME, mappings=mappings, settings=settings)

In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df)):
    es.index(index="stories", document=row.to_dict())